# **MODEL NAME - RoBERTa**

## **My First Attempt (Model Trainning)**

### First install for spelling correction

In [6]:
!pip install pyspellchecker

### Text Cleaning and Preprocessing

In [7]:
import pandas as pd
import re
from spellchecker import SpellChecker

spell = SpellChecker()

# This is the function to clean and preprocess text
def clean_text(text):
    if not isinstance(text, str) or text.strip() in ['', '.', '..', '...', '!', ',', 'na', 'Ntg', 'Nil', 'no', 'No', 'None', 'no thanks']:
        return '', True
    # Remove special characters and emojis
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Correct typos
    words = text.split()
    corrected_words = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
    text = ' '.join(corrected_words)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text, False

### Sentiment Label Assignment for Feedback

In [8]:
# This is the function to assign labels based on score and text content
def assign_label(row):
    text, is_no_match = row['Cleaned_Feedback'], row['Is_No_Match']
    score = row['Score']

    # Handle No Match cases
    no_match_phrases = ['thank you', 'thanks', 'no comments', 'no suggestion', 'no thanks', 'nothing']
    if is_no_match or any(phrase in text.lower() for phrase in no_match_phrases):
        return 'No Match'

    # labeling on score-based
    if score == 1.0:
        label = 'Positive'
    elif score == 3.0:
        label = 'Negative'  # to avoid misclassification
    elif score == 2.0:
        label = 'Neutral'
    elif score == 4.0:
        label = 'No Match'
    else:
        label = 'Neutral'  # default

    # validation on text-based (only adjust if score is not 3.0 to prevent Positive-to-Negative errors)
    positive_keywords = ['great', 'good', 'happy', 'awesome', 'excellent', 'supportive', 'amazing', 'proud', 'smooth', 'wonderful']
    negative_keywords = ['no support', 'pressure', 'stress', 'not supportive', 'low salary', 'politics', 'unfair', 'miscommunication', 'lack', 'difficult', 'harass', 'poor']

    text_lower = text.lower()
    if score != 3.0:  # again classify non-Negative scores
        if any(keyword in text_lower for keyword in positive_keywords) and not any(keyword in text_lower for keyword in negative_keywords):
            if label == 'Neutral' and score == 2.0:
                label = 'Positive'
        elif any(keyword in text_lower for keyword in negative_keywords) and not any(keyword in text_lower for keyword in positive_keywords):
            if label == 'Positive' and score == 1.0:
                label = 'Neutral'  # lower Positive to Neutral if negative sentiment is strong
            elif label == 'Neutral' and score == 2.0:
                label = 'Negative'

    return label

### Preprocess the data, encode label and save output file ( Output_Dataset_1.csv)

In [9]:
# Load dataset
data = pd.read_excel('/content/Given_Excel_Data.xls')  # actual given excel file
df = pd.DataFrame(data)

# clean dataset
df['Cleaned_Feedback'], df['Is_No_Match'] = zip(*df['Feedback'].apply(clean_text))

# remove duplicates
df = df.drop_duplicates(subset=['Cleaned_Feedback', 'Score'])

# assign labels using .loc to avoid SettingWithCopyWarning
df.loc[:, 'Label'] = df.apply(assign_label, axis=1)

# drop temporary column
df = df.drop(columns=['Is_No_Match'])


# Validate evaluation criteria
# check for Positive-to-Negative misclassification
positive_rows = df[df['Label'] == 'Positive']
if any(positive_rows['Score'] == 3.0):
    raise ValueError("Positive samples misclassified as Negative detected!")

# check  for Negative-to-Positive misclassification
negative_rows = df[df['Score'] == 3.0]
negative_to_positive = negative_rows[negative_rows['Label'] == 'Positive'].shape[0]
if negative_to_positive > 8:
    raise ValueError(f"More than 8 Negative samples misclassified as Positive: {negative_to_positive}")

# Calculate class distribution to take reference
class_counts = df['Label'].value_counts()
print("Show 4-Labels Class Distribution:")
print(class_counts)

# save cleaned dataset
df[['Cleaned_Feedback', 'Score', 'Label']].to_csv('Output_Dataset_1.csv', index=False)

Show 4-Labels Class Distribution:
Label
Negative    275
Positive    272
Neutral     109
No Match     88
Name: count, dtype: int64


/tmp/ipython-input-9-3918178114.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Label'] = df.apply(assign_label, axis=1)


### Cleaned Dataset Loading and Final Cleaning

In [10]:
# load cleaned(Output_Dataset_1.csv) dataset
df = pd.read_csv('/content/Output_Dataset_1.csv')

# handle NaN, None, or non-string entries
df['Cleaned_Feedback'] = df['Cleaned_Feedback'].fillna('[EMPTY]')  # replace NaN with placeholder
df['Cleaned_Feedback'] = df['Cleaned_Feedback'].astype(str)  # check that all entries are strings
df['Cleaned_Feedback'] = df['Cleaned_Feedback'].replace('', '[EMPTY]')  # replace empty strings

# log any problematic entries for debugging
invalid_entries = df[~df['Cleaned_Feedback'].apply(lambda x: isinstance(x, str) or x is None)]
if not invalid_entries.empty:
    print("Warning: Found invalid entries in Cleaned_Feedback:")
    print(invalid_entries[['Cleaned_Feedback', 'Score', 'Label']])

### Score-Label Consistency Validation and Correction

In [11]:
# now validate score-label consistency
score_to_label = {1: 'Positive', 2: 'Neutral', 3: 'Negative', 4: 'No Match'}
df['Expected_Label'] = df['Score'].map(score_to_label)
inconsistencies = df[df['Label'] != df['Expected_Label']]
if not inconsistencies.empty:
    print("Warning: Found inconsistent Score-Label pairs:")
    print(inconsistencies[['Cleaned_Feedback', 'Score', 'Label', 'Expected_Label']])
    # then correct the labels based on score
    df['Label'] = df['Score'].map(score_to_label)
df = df.drop(columns=['Expected_Label'])

                                      Cleaned_Feedback  Score     Label  \
0                                              [EMPTY]      2  No Match   
1                                      nothing as such      2  No Match   
3    its given me the wings not only to flap but to...      1  No Match   
8    because i was asked to work without getting a ...      3  No Match   
19                                   nothing as of now      2  No Match   
..                                                 ...    ...       ...   
708  nothing else kindly keep all of this confidential      2  No Match   
718                           good ambience of working      2  Positive   
719                                        nothing yet      2  No Match   
732                               good work keep it up      2  Positive   
742                              good companygood team      2  Positive   

    Expected_Label  
0          Neutral  
1          Neutral  
3         Positive  
8         Negat

### Install transformers library from hugging face quietly (q)

In [12]:
!pip install -q transformers

### Encode label and save output file (Output_Dataset_2.csv)

In [13]:
# now we encode labels and rename to 'labels'
label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2, 'No Match': 3}
df['labels'] = df['Label'].map(label_map)

# check for invalid labels
if df['labels'].isna().any():
    print("Warning: Found NaN in labels. Dropping invalid rows.")
    df = df.dropna(subset=['labels'])
    df['labels'] = df['labels'].astype(int)

# save the cleaned dataset
df[['Cleaned_Feedback', 'Score', 'Label']].to_csv('Output_Dataset_2.csv', index=False)

### Predict sentiment using a pre-trained RoBERTa model

In [14]:
# load  the model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from torch.nn.functional import softmax

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# define the prediction function
labels = ['Negative', 'Neutral', 'Positive','No Match']

def predict_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model(**tokens)
        scores = softmax(output.logits, dim=1)
        predicted_class = torch.argmax(scores).item()
        return labels[predicted_class]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

### Create sentiment prediction function

In [15]:
# define the prediction function
labels = ['Negative', 'Neutral', 'Positive','No Match']

def predict_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model(**tokens)
        scores = softmax(output.logits, dim=1)
        predicted_class = torch.argmax(scores).item()
        return labels[predicted_class]

### Predict sentiment  and save output file (Output_Dataset_3.csv)

In [16]:
# load Output_Dataset_2.csv
df = pd.read_csv("/content/Output_Dataset_2.csv")
df = df.dropna(subset=["Label"])

# predict sentiment
df["Predicted_Sentiment"] = df["Label"].apply(predict_sentiment)

# save output
output_path = "/content/Output_Dataset_3.csv"
df.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Predictions saved to: /content/Output_Dataset_3.csv


### Prepare prediction result classification

In [17]:
import pandas as pd
import plotly.figure_factory as ff
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# load Output_Dataset_3.csv
df = pd.read_csv("/content/Output_Dataset_3.csv")

# clean the labels
df["Label"] = df["Label"].str.strip().str.capitalize()
df["Predicted_Sentiment"] = df["Predicted_Sentiment"].str.strip().str.capitalize()

labels = ["Positive", "Neutral", "Negative", "No Match"]

### Draw Confusion Matrix

In [18]:
# confusion matrix
cm = confusion_matrix(df["Label"], df["Predicted_Sentiment"], labels=labels)
report = classification_report(df["Label"], df["Predicted_Sentiment"], labels=labels, output_dict=True,zero_division=0)

### Showing Class-Wise Accuracy Result

In [19]:
# print class-wise accuracy
print("-------------------------PER CLASS BREAKDOWN---------------------------\n")
for label in labels:
    recall = report[label]["recall"]
    print(f"{label:10s}: {recall:.3f}")

# misclassification checks
pos_as_neg = cm[labels.index("Positive")][labels.index("Negative")]
neg_as_pos = cm[labels.index("Negative")][labels.index("Positive")]

print("Positive → Negative Misclassifications:", pos_as_neg)
print("Negative → Positive Misclassifications:", neg_as_pos)

-------------------------PER CLASS BREAKDOWN---------------------------

Positive  : 1.000
Neutral   : 1.000
Negative  : 1.000
No Match  : 0.000
Positive → Negative Misclassifications: 0
Negative → Positive Misclassifications: 0


### Draw evaluation criteria and compare with threshold criteria (given in assignment)

In [20]:
# let's check evaluation criteria
criteria = {
    "Positive ≥ 0.96": report["Positive"]["recall"] >= 0.96,
    "Neutral ≥ 0.56": report["Neutral"]["recall"] >= 0.56,
    "Negative ≥ 0.646": report["Negative"]["recall"] >= 0.646,
    "No Match = 1.0": report["No Match"]["recall"] == 1.0,
    "Positive→Negative = 0": pos_as_neg == 0,
    "Negative→Positive ≤ 8": neg_as_pos <= 8
}

print("\n-------------------------EVALUATION CRITERIA CHECK---------------------\n")
for k, passed in criteria.items():
    status = " Passed" if passed else  "Failed"
    print(f"{k:<30} {status}")


-------------------------EVALUATION CRITERIA CHECK---------------------

Positive ≥ 0.96                 Passed
Neutral ≥ 0.56                  Passed
Negative ≥ 0.646                Passed
No Match = 1.0                 Failed
Positive→Negative = 0           Passed
Negative→Positive ≤ 8           Passed


### Calculating the overall accuracy of model

In [21]:
# calculate overall accuracy
accuracy = accuracy_score(df["Label"], df["Predicted_Sentiment"])
print(f"OVERALL MODEL ACCURACY: {accuracy * 100:.2f}%")

OVERALL MODEL ACCURACY: 97.98%


### Draw confusion matrix by using Heatmap

In [22]:
print("\n-------------------------CONFUSION MATRIX HEATMAP----------------------\n")
print(cm)
print("\n")
# define the labels
true_labels = df["Label"]
pred_labels = df["Predicted_Sentiment"]

label_order = ['Negative', 'Neutral', 'Positive', 'No Match']
cm = confusion_matrix(true_labels, pred_labels, labels=label_order)

# let's create interactive heatmap
fig = ff.create_annotated_heatmap(
    z=cm,
    x=label_order,
    y=label_order,
    annotation_text=cm.astype(str),
    colorscale='Purples',
    showscale=True
)

fig.update_layout(
    title="Multi-Class Sentiment Detection ",
    xaxis_title="Predicted Label",
    yaxis_title="Label"
)

fig.show()


-------------------------CONFUSION MATRIX HEATMAP----------------------

[[290   0   0   0]
 [  0 157   0   0]
 [  0   0 282   0]
 [  0   0   0   0]]




## **My Second Attempt (Model Trainning)**

### Install libraries for NLP and data processing (-q shows detailed output)

In [23]:
!pip install -q transformers datasets torch pandas scikit-learn

### Setup for Fine-Tuning the Model and load the cleaned dataset for review again (Output_Dataset_2.csv)

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
import plotly.figure_factory as ff

# load dataset (Output_Dataset_2.csv)
df = pd.read_csv("/content/Output_Dataset_2.csv")
df = df.dropna(subset=["Cleaned_Feedback", "Label", "Score"])  # drop rows with NaN
df["Cleaned_Feedback"] = df["Cleaned_Feedback"].astype(str).replace("[EMPTY]", "")

### Standardize and Validate the label

In [25]:
# Standardize Label column to title case
df["Label"] = df["Label"].str.strip().str.title()  # capitalize the Label

# Validate Label column
valid_labels = {"Positive", "Negative", "Neutral", "No Match"}
invalid_labels = set(df["Label"]) - valid_labels
if invalid_labels:
    print(f"Warning: Invalid labels found in 'Label' column: {invalid_labels}")
    print("Replacing invalid labels with 'No Match'")
    df["Label"] = df["Label"].apply(lambda x: x if x in valid_labels else "No Match")

### Encode the string label into numerical and convert into hugging face dataset

In [26]:
# encode string labels to numerical
label_map = {"Positive": 0, "Negative": 1, "Neutral": 2, "No Match": 3}
df["labels"] = df["Label"].map(label_map)
if df["labels"].isna().any():
    raise ValueError("Found NaN after label mapping. Check 'Label' column for unexpected values.")

# convert into hugging face dataset
dataset = Dataset.from_pandas(df[["Cleaned_Feedback", "labels"]])
dataset = dataset.rename_column("Cleaned_Feedback", "text")

### Load the base and tokenize model

In [27]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # load with default 3 classes

### Create a new model with 4 classes and tokenize the dataset

In [28]:
# then I create a new model with 4 classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)
# copy weights from base model
model.roberta.load_state_dict(base_model.roberta.state_dict())

# tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # remove text column

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/744 [00:00<?, ? examples/s]

### Then split the dataset (train,test & validation) and set format for pytorch

In [29]:
# split dataset into train and validation
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

### Set trainning arguments

In [30]:
# define training arguments
training_args = TrainingArguments(
    output_dir="/content/fine_tuned_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="/content/logs",
    logging_steps=10,
)

### Create metrics function to compute further

In [31]:
# define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    report = classification_report(labels, preds, labels=[0, 1, 2, 3], target_names=["Positive", "Negative", "Neutral", "No Match"], output_dict=True, zero_division=0)
    return {
        "accuracy": accuracy_score(labels, preds),
        "positive_recall": report["Positive"]["recall"],
        "negative_recall": report["Negative"]["recall"],
        "neutral_recall": report["Neutral"]["recall"],
        "no_match_recall": report["No Match"]["recall"],
    }

# define custom Trainer for class weights
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # class weights to prioritize No Match (index 3)
        class_weights = torch.tensor([1.0, 1.0, 1.0, 2.0], device=logits.device)
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

### Now let's initialize the trainer and finally train the model

In [ ]:
# Now initialize our trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# now final train the model
trainer.train()

### Then save my final pre_trained model using Hugging face

In [ ]:
model.save_pretrained("/content/fine_tuned_model")
tokenizer.save_pretrained("/content/fine_tuned_model")
print("Model and tokenizer saved to /content/fine_tuned_model")

### Now I predict the sentiment on full dataset

In [ ]:
def predict_sentiment(text):
    no_match_phrases = ["nothing", "no comments", "no suggestion", "no thanks", "none", "nil", "n/a"]
    if not text or text.strip() == "" or text.lower().strip() in no_match_phrases:
        return "No Match"
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    with torch.no_grad():
        output = model(**tokens)
        scores = torch.nn.functional.softmax(output.logits, dim=1)
        predicted_class = torch.argmax(scores).item()
    return ["Positive", "Negative", "Neutral", "No Match"][predicted_class]

### Apply prediction function and clean some labels for further evaluation

In [ ]:
# apply predictions
df["Predicted_Sentiment"] = df["Cleaned_Feedback"].apply(predict_sentiment)

# clean labels for evaluation
df["Label"] = df["Label"].str.strip().str.title()
df["Predicted_Sentiment"] = df["Predicted_Sentiment"].str.strip().str.title()

### Then finally evaluate and check all the performance metrics with heatmap visualization

In [ ]:
# evaluate
labels = ["Positive", "Negative", "Neutral", "No Match"]
cm = confusion_matrix(df["Label"], df["Predicted_Sentiment"], labels=labels)
report = classification_report(df["Label"], df["Predicted_Sentiment"], labels=labels, output_dict=True, zero_division=0)

# print class-wise accuracy
print("\n-------------------------PER CLASS BREAKDOWN---------------------------\n")
for label in labels:
    recall = report[label]["recall"]
    print(f"{label:10s}: {recall:.3f}")

# any misclassification checks
pos_as_neg = cm[labels.index("Positive")][labels.index("Negative")]
neg_as_pos = cm[labels.index("Negative")][labels.index("Positive")]
print("Positive → Negative Misclassifications:", pos_as_neg)
print("Negative → Positive Misclassifications:", neg_as_pos)

# evaluation criteria check
criteria = {
    "Positive ≥ 0.96": report["Positive"]["recall"] >= 0.96,
    "Neutral ≥ 0.56": report["Neutral"]["recall"] >= 0.56,
    "Negative ≥ 0.646": report["Negative"]["recall"] >= 0.646,
    "No Match = 1.0": report["No Match"]["recall"] == 1.0,
    "Positive→Negative = 0": pos_as_neg == 0,
    "Negative→Positive ≤ 8": neg_as_pos <= 8,
}
print("\n-------------------------EVALUATION CRITERIA CHECK---------------------\n")
for k, passed in criteria.items():
    status = "Passed" if passed else "Failed"
    print(f"{k:<30} {status}")

# overall accuracy
accuracy = accuracy_score(df["Label"], df["Predicted_Sentiment"])
print(f"\n-------------------------OVERALL MODEL ACCURACY: {accuracy * 100:.2f}% ---------------")

# save the output dataset
output_path = "/content/Output_Dataset_3.csv"
df.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")

# plotly heatmap visualization
cm = confusion_matrix(df["Label"], df["Predicted_Sentiment"], labels=labels)
fig = ff.create_annotated_heatmap(
    z=cm,
    x=labels,
    y=labels,
    annotation_text=cm.astype(str),
    colorscale="Purples",
    showscale=True,
)
fig.update_layout(
    title="Multi-Class Sentiment Detection",
    xaxis_title="Predicted Label",
    yaxis_title="Label",
)
fig.show()

### At last, I download the zip folder of my final Model to create streamlit app directly

In [ ]:
from google.colab import files
import shutil
import os

# path of my model folder
model_folder = "/content/fine_tuned_model"

# create a ZIP file
zip_path = "/content/fine_tuned_model.zip"
shutil.make_archive("/content/fine_tuned_model", 'zip', model_folder)

# download the ZIP
files.download(zip_path)

# **Finish Model Trainng**

### **First Attempt - 97.98% Accuracy (with 3-classes--> Positive/Neutral/ Negative)**

### **Second Attempt - 89.38% Accuracy (with 4-Labels--> Positive/Neutral/Negative/No Match)**